## Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Didattica/ML/exam_2024-2025/project_work_segmentazione_off-road/")

## General Setup

In [ ]:

from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large, DeepLabV3_MobileNet_V3_Large_Weights
from torch import nn
import os
import torch
import albumentations as A

# Class Mapper
COLOR_TO_ID = {
    (255, 255, 255): 0,   # Background
    (178, 176, 153): 1,   # Smooth Trail
    (128, 255, 0):   2,   # Traversable grass
    (156, 76, 30):   3,   # Rough Trail
    (255, 0, 128):   4,   # Puddle
    (255, 0, 0):     5,   # Obstacle
    (0, 160, 0):     6,   # Non Traversable Low Vegetation
    (40, 80, 0):     7,   # High Vegetation
    (1, 88, 255):    8    # Sky
}

IMG_HEIGHT = 512
IMG_WIDTH = 512
TEST_DIR = './resources/test'       # Change to the correct directory
MODEL_SAVE_PATH = 'best_segmentation_model.pth'

## Model Architecture

In [ ]:

class SegmentationModel(nn.Module):
    def __init__(self, num_classes=9):
        super(SegmentationModel, self).__init__()

        self.model = deeplabv3_mobilenet_v3_large(weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1)

        self.model.classifier[-1] = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, x):
        return self.model(x)['out']

model_summary = SegmentationModel()
print(model_summary)

## Function to plot correctly

In [ ]:
ID_TO_COLOR = {v: k for k, v in COLOR_TO_ID.items()}

# This feature can be used to display images correctly by mapping the various pixels.
def id_to_rgb_mask(id_mask_np, id_to_color_map):
    h, w = id_mask_np.shape
    rgb_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color_rgb in id_to_color_map.items():
        rgb_mask[id_mask_np == class_id] = color_rgb
    return rgb_mask

## Function to load the model

In [ ]:

def load_model(path, device='cuda'):
    model = SegmentationModel().to(device)
    model.load_state_dict(torch.load(path, weights_only=True))
    return model

## Function to predict on a single image

In [ ]:
img_transform = A.Compose([
    A.Resize(height=IMG_HEIGHT, width=IMG_WIDTH),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.ToTensorV2()
])

def pil_to_tensor(img):
    img = np.array(img)
    img = img_transform(image=img)['image']
    return img

def predict(model, image, target_shape, device='cuda'):
    model.eval()
    with torch.no_grad():
        image = Image.fromarray(image)
        image_tensor = pil_to_tensor(image).unsqueeze(0).to(device)
        output = model(image_tensor)
        predictions = torch.nn.functional.softmax(output, dim=1)
        pred_labels = torch.argmax(predictions, dim=1)
        pred_labels = pred_labels.squeeze(0).cpu().numpy().astype(np.uint8)
        if target_shape is not None and pred_labels.shape != target_shape:
            pred_labels = np.array(Image.fromarray(pred_labels).resize((target_shape[1], target_shape[0]), resample=Image.NEAREST))
    return pred_labels

## Code to test the model

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Metrics
def compute_iou(mask1, mask2, label):
  intersection = np.sum((mask1 == label) & (mask2 == label))
  union = np.sum((mask1 == label) | (mask2 == label))
  if union == 0:
    return np.nan
  return intersection / union
def compute_all_iou(mask1, mask2, num_labels=8):
  iou_scores = np.zeros((num_labels))
  for label in range(num_labels):
    iou = compute_iou(mask1, mask2, label+1) # we skip the background label
    iou_scores[label] = iou
  return iou_scores


# Run YOUR LOAD_MODEL FUNCTION
model = load_model(path=MODEL_SAVE_PATH)

# Main loop
test_dir = TEST_DIR  # we will change this path with that of the private test set directory
samples = os.listdir(test_dir)
IOUs = np.zeros((len(samples), 8))
verbose = True

for i, subdir in tqdm(enumerate(samples), desc="Processing samples"):
    subdir_path = os.path.join(test_dir, subdir)

    if os.path.isdir(subdir_path):
        # Get the data paths
        rgb_path = os.path.join(subdir_path, 'rgb.jpg')
        labels_path = os.path.join(subdir_path, 'labels.png')

        if os.path.exists(rgb_path) and os.path.exists(labels_path):
            if verbose:
                print(f"Processing subdirectory: {subdir}")

            try:  # ATTENTION: any error occurring in this try-catch means that the corresponding IOUs are evaluated as ZERO

                # Open images
                rgb_image = Image.open(rgb_path)
                rgb_array = np.asarray(rgb_image).copy()
                labels_image = Image.open(labels_path).copy()
                labels_array = np.asarray(labels_image)
                if verbose:
                    print(f"  Loaded {rgb_path} and {labels_path}")

                # Run YOUR PREDICT FUNCTION
                predicted_labels_array = predict(model=model, image=rgb_array, target_shape=labels_array.shape)

                # Evaluate the IOU metric
                IOUs[i,:] = compute_all_iou(labels_array, predicted_labels_array)

                if verbose:
                    labels_vals = np.unique(np.asarray(labels_image))
                    print(f"  Unique labels values: {labels_vals}")
                    predicted_labels_vals = np.unique(np.asarray(predicted_labels_array))
                    print(f"  Unique predicted labels values: {predicted_labels_vals}")

                    plt.subplot(1, 3, 1)
                    plt.imshow(rgb_image)
                    plt.subplot(1, 3, 2)
                    plt.imshow(labels_image)
                    plt.subplot(1, 3, 3)
                    plt.imshow(predicted_labels_array)
                    #plt.imshow(id_to_rgb_mask(predicted_labels_array, ID_TO_COLOR))
                    plt.show()

                rgb_image.close()
                labels_image.close()

            except FileNotFoundError:
                print(f"  Error: Could not find image files in {subdir_path}")
            except Exception as e:
                print(f"  Error processing images in {subdir_path}: {e}")
        else:
            print(f"  Skipping subdirectory {subdir}: rgb.jpg or labels.png not found.")

score = np.nanmean(IOUs)
print(f"\nFinal competition score: {score}")

## More information useful for your analysis

In [ ]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)
print(f"All IOUs:\n{IOUs}")
print("Average IOUs for each:")
print(f"- class: {np.nanmean(IOUs, 0)}")
print(f"- image: {np.nanmean(IOUs, 1)}")